In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from typing import NamedTuple, Optional


In [ ]:
#MDPの定義
class MDP(NamedTuple):
    S_array :np.array
    A_array :np.array
    gamma : float
    horizon : int
    rew : float
    P : float

    def S(self) -> int:
        return len(self.S_array)

    def A(self) -> int:
        return len(self.A_array)


In [ ]:
mdp = MDP
S = 2
A = 3
P= np.zeros((S,A,S))
P[0][0][0] = 0.8
P[0][0][1] = 0.2
P[0][1][1] = 1
P[0][2][0] = 1
P[1][0][1] = 1

R = np.zeros((S,A))
R[0][0] = 5
R[0][1] = 10
R[0][2] = 2
R[1][0] = 1


random_policy= np.zeros((S,A))
#S=0のときは必ず行動0,S=1のときも必ず行動0を出す決定的方策
random_policy[0][0] = 1
random_policy[1][0] = 1

def choose_element_with_probability(probabilities):
    elements = [0, 1]
    selected_element = random.choices(elements, probabilities)[0]
    return selected_element

def sample_action_from_policy(s,policy):
    return np.argmax(policy[s][:])

def sample_state_from_action(s,a,P):
    if np.max(P[s][a][:]) == 1:
        return np.argmax(P[s][a][:])
    elif np.max(P[s][a][:]) == 0:
        return s
    else:
        return choose_element_with_probability(P[s][a][:])   


#step関数でs,a,r,s'を求める
def step(policy,P,R,state):
    action = sample_action_from_policy(state,policy)
    next_state = sample_state_from_action(state,action,P)
    reward = R[state][action]
    SARS = state,action,reward,next_state
    return SARS


#s=0からはじめて100回policyにしたがって行動したデータを収集
state = 0
data = []
sars = step(random_policy,P,R,state)
data.append(sars)
for i in range(10000):
    sars = step(random_policy,P,R,sars[3])
    data.append(sars)


In [ ]:
gamma = 0.9


In [109]:
#データからモンテカルロ推定する
T_dash = 9000

def calc_state_appearance(state,data,num):
    state_ap_num = 0
    ap_idx = []
    for i in range(num):
        if data[i][0] == state:
            state_ap_num += 1
            ap_idx.append(i)
    return state_ap_num,ap_idx

num,ap_idx = calc_state_appearance(1,data,10000)

def calc_reward_from_state_as_s0(state,data,num):
    c = 0
    
def monte_carlo(state,data,num,gamma):
    reward = 0
    rewards = 0
    multiplication_num = 1
    state_ap_num = 0
    for i in range(num):
         if data[i][0] == state:
            state_ap_num += 1
            for j in range(i,num):
                reward += data[j][2] * gamma ** multiplication_num
                multiplication_num += 1
            rewards += reward

    return rewards/state_ap_num

monte_carlo(0,data,10000,gamma) #πに従う行動履歴から推定した状態0の価値
monte_carlo(0,data,10000,gamma) #πに従う行動履歴から推定した状態1の価値

18.756000000000018